# BIDS filenames and path construction

You may want to create valid BIDS filenames for files that are new or hypothetical that would sit within your BIDS project. 

This is useful when you know what entity values you need to write out to, but don't want to deal with looking up the precise BIDS file-naming syntax. 

In the example below, imagine we've created a new file containing stimulus presentation information, and we want to save it to a `.tsv.gz` file, per the BIDS naming conventions. 

All we need to do is define a dictionary with the name components, and `build_path` takes care of the rest (including injecting sub-directories!):


In [9]:
# lint with black
%load_ext lab_black

In [8]:
from bids import BIDSLayout
from bids.tests import get_test_data_path
import os

In [16]:
# Here we're using an example BIDS dataset that's bundled with the pybids tests
data_path = os.path.join(get_test_data_path(), "7t_trt")

# Initialize the layout
raw = BIDSLayout(data_path)
raw

BIDS Layout: .../pybids/bids/tests/data/7t_trt | Subjects: 10 | Sessions: 20 | Runs: 20

Let's pretend we are about to create some derivatives data for that dataset.

In [11]:
pipeline_name = "pybids"
derivative_dir = os.path.join(raw.root, "derivatives", pipeline_name)

if not os.path.exists(derivative_dir):
    os.makedirs(derivative_dir)
derivative = BIDSLayout(derivative_dir, validate=False)
derivative

BIDS Layout: ...data/7t_trt/derivatives/pybids | Subjects: 0 | Sessions: 0 | Runs: 0

In [43]:
bf = raw.get(subject="01", session="1", run=1, suffix="bold", extension="nii.gz")
entities = bf[0].get_entities()
entities

{'acquisition': 'fullbrain',
 'datatype': 'func',
 'extension': '.nii.gz',
 'run': 1,
 'session': '1',
 'subject': '01',
 'suffix': 'bold',
 'task': 'rest'}

In [13]:
raw.build_path(entities)

'/home/remi/github/pybids/bids/tests/data/7t_trt/sub-01/ses-1/func/sub-01_ses-1_task-rest_acq-fullbrain_run-1_bold.nii.gz'

You can also use `build_path` in more sophisticated ways—for example, by defining your own set of matching templates that cover cases not supported by BIDS out of the box. 

For example, suppose you want to create a template for naming a new z-stat file. 

You could do something like:

In [54]:
# Define the pattern to build out of the components passed in the dictionary
filename_pattern = (
    "sub-{subject}"
    + "[_ses-{session}]"
    + "_task-{task}"
    + "[_acq-{acquisition}]"
    + "[_rec-{reconstruction}]"
    + "[_run-{run}]"
    + "[_echo-{echo}]"
    + "[_desc-{desc}]"
    + "_{suffix}.{extension}"
)
pattern = os.path.join("sub-{subject}", "[ses-{session}]", filename_pattern)
print(pattern)

entities["desc"] = "mean"

# Notice we pass the new pattern as the second argument
derivative.build_path(entities, pattern, validate=False)

sub-{subject}/[ses-{session}]/sub-{subject}[_ses-{session}]_task-{task}[_acq-{acquisition}][_rec-{reconstruction}][_run-{run}][_echo-{echo}][_desc-{desc}]_{suffix}.{extension}


'/home/remi/github/pybids/bids/tests/data/7t_trt/derivatives/pybids/sub-01/ses-1/sub-01_ses-1_task-rest_acq-fullbrain_run-1_desc-mean_bold.nii.gz'

Note that in the above example, we set `validate=False` to ensure that the standard BIDS file validator doesn't run (because the pattern we defined isn't actually compliant with the BIDS specification).

### Writing to files

In [1]:
def set_dataset_description(is_derivative=True):

    content = {
        "Name": "dataset name",
        "BIDSVersion": "1.6.0",
        "DatasetType": "raw",
        "License": "",
        "Authors": ["", ""],
        "Acknowledgements": "Special thanks to ",
        "HowToAcknowledge": "Please cite this paper: ",
        "Funding": ["", ""],
        "EthicsApprovals": [""],
        "ReferencesAndLinks": ["", ""],
        "DatasetDOI": "doi:",
        "HEDVersion": "",
    }

    if is_derivative:
        content["GeneratedBy"] = [
            {
                "Name": "",
                "Version": "",
                "Container": {"Type": "", "Tag": ""},
                "Description": "",
                "CodeURL": "",
            },
        ]

        content["SourceDatasets"] = [
            {
                "DOI": "doi:",
                "URL": "",
                "Version": "",
            }
        ]

    return content

In [37]:
import json
from bids.layout.writing import write_to_file

In [39]:
entities = {"suffix": "dataset_description", "extension": ".json"}

content = set_dataset_description()

# derivative.write_to_file(entities, json.dumps(content), validate=False, strict=False)

write_to_file(
    root=derivative.root, path="dataset_description.json", contents=json.dumps(content)
)


### TODO

write json for derivatives

## Other utilities

### Filename parsing
Say you have a filename, and you want to manually extract BIDS entities from it. The `parse_file_entities` method provides the facility:

In [20]:
path = "/a/fake/path/to/a/BIDS/file/sub-01_run-1_T2w.nii.gz"
raw.parse_file_entities(path)

{'subject': '01', 'run': 1, 'suffix': 'T2w', 'extension': '.nii.gz'}

A version of this utility independent of a specific layout is available at `bids.layout` ([doc](https://bids-standard.github.io/pybids/generated/bids.layout.parse_file_entities.html#bids.layout.parse_file_entities)).

In [21]:
from bids.layout import parse_file_entities

path = "/a/fake/path/to/a/BIDS/file/sub-01_run-1_T2w.nii.gz"
parse_file_entities(path)

{'subject': '01', 'run': 1, 'suffix': 'T2w', 'extension': '.nii.gz'}